# <center> Лабораторная работа 3.  </center>
## <center> Применение свёрточных нейронных сетей в задаче классификации изображений</center>
## <center> Ваганова А.И., группа 20223 </center>

## Описание задачи 
### На основании предложенных примеров изучить сверточные нейронные сети для классификации изображений по базам MNIST и CIFAR10.

In [ ]:
from tensorflow.keras import datasets, Sequential, layers, callbacks
import numpy as np
import time

In [ ]:
(X, y), (X_test, y_test) = datasets.mnist.load_data()
mnist = (X[..., None]/128 - 1, y)
(X, y), (X_test, y_test) = datasets.cifar10.load_data()
cifar = cifar = (X/128 - 1, y)

170508288/170498071 [==============================] - 2s 0us/step




---





---



In [ ]:
def get_model(n_conv_layers=6, n_dense_layers=0, n_filters=32, act_fn='relu', pool='max'):
  model = Sequential()
  for i in range(n_conv_layers):
    if i>0 and i%2 == 0:
      model.add(layers.AvgPool2D() if pool=='avg' else layers.MaxPool2D())
    model.add(layers.Conv2D(n_filters, kernel_size=3, padding='same', activation=act_fn))
  model.add(layers.GlobalAvgPool2D() if pool=='avg' else layers.GlobalMaxPool2D())
  for i in range(n_dense_layers):
    model.add(layers.Dense(1000, activation=act_fn))
  model.add(layers.Dense(10, 'softmax'))
  model.compile('adam', 'sparse_categorical_crossentropy', 'accuracy')
  return model

In [ ]:
def train_model(dataset, model, verbose=0):
  start_time = time.time()
  callbacks_list = [callbacks.EarlyStopping(patience=5),
                    callbacks.ReduceLROnPlateau(patience=3, min_lr=1e-5)]
  model.fit(*dataset, validation_split=1/2, batch_size=256, epochs=100,
            callbacks=callbacks_list, verbose=verbose)
  elapsed_time = time.time() - start_time
  return np.max(model.history.history['val_accuracy']), elapsed_time

# MNIST

#### Точность классификации в зависимости от числа сверточных слоев

In [ ]:
for n_conv_layers in range(1, 11):
  acc, elapsed = train_model(mnist, get_model(n_conv_layers=n_conv_layers))
  print(f'{n_conv_layers} conv layers, time {elapsed:.0f} sec, accuracy {acc:.4f}')

1 conv layers, time 82 sec, accuracy 0.7428  
2 conv layers, time 115 sec, accuracy 0.9430  
3 conv layers, time 35 sec, accuracy 0.9767  
4 conv layers, time 62 sec, accuracy 0.9844  
5 conv layers, time 29 sec, accuracy 0.9859  
6 conv layers, time 27 sec, accuracy 0.9860  
7 conv layers, time 28 sec, accuracy 0.9892  
8 conv layers, time 26 sec, accuracy 0.9894  
9 conv layers, time 36 sec, accuracy 0.9900  
10 conv layers, time 30 sec, accuracy 0.9863  

Вывод: увеличение количества сверточных слоев повышает точность (только на 10 слоях наблюдается падение).

#### Точность классификации в зависимости от числа каналов

In [ ]:
for n_filters in np.logspace(1, 2, num=10, dtype=int):
  acc, elapsed = train_model(mnist, get_model(n_filters=n_filters))
  print(f'{n_filters} filters, time {elapsed:.0f} sec, accuracy {acc:.4f}')

10 filters, time 56 sec, accuracy 0.9769  
12 filters, time 37 sec, accuracy 0.9801  
16 filters, time 39 sec, accuracy 0.9837  
21 filters, time 32 sec, accuracy 0.9849  
27 filters, time 31 sec, accuracy 0.9857  
35 filters, time 33 sec, accuracy 0.9874  
46 filters, time 31 sec, accuracy 0.9884  
59 filters, time 38 sec, accuracy 0.9894  
77 filters, time 46 sec, accuracy 0.9907  
100 filters, time 62 sec, accuracy 0.9906  


Вывод: увеличение количества каналов повышает точность. В этом эксперименте каждый слой имел одинаковое кол-во каналов.

#### Точность классификации в зависимости от типа пулинга

In [ ]:
for pool in ['max', 'avg']:
  for i in range(3):
    acc, elapsed = train_model(mnist, get_model(pool=pool))
    print(f'{pool} pooling, time {elapsed:.0f} sec, accuracy {acc:.4f}')

max pooling, time 29 sec, accuracy 0.9876  
max pooling, time 29 sec, accuracy 0.9872  
max pooling, time 36 sec, accuracy 0.9876  
avg pooling, time 70 sec, accuracy 0.9850  
avg pooling, time 56 sec, accuracy 0.9754  
avg pooling, time 88 sec, accuracy 0.9848  

Вывод: в этом эксперименте max pooling оказался лучше average pooling.

#### Точность классификации в зависимости от функции активации

In [ ]:
for act_fn in ['relu', 'sigmoid', 'tanh', 'softplus', 'linear']:
  for i in range(3):
    acc, elapsed = train_model(mnist, get_model(act_fn=act_fn))
    print(f'{act_fn} activation, time {elapsed:.0f} sec, accuracy {acc:.4f}')

relu activation, time 42 sec, accuracy 0.9863  
relu activation, time 27 sec, accuracy 0.9870  
relu activation, time 26 sec, accuracy 0.9868  
sigmoid activation, time 203 sec, accuracy 0.9764  
sigmoid activation, time 143 sec, accuracy 0.9830  
sigmoid activation, time 145 sec, accuracy 0.9835  
tanh activation, time 42 sec, accuracy 0.9899  
tanh activation, time 36 sec, accuracy 0.9880  
tanh activation, time 45 sec, accuracy 0.9889  
softplus activation, time 38 sec, accuracy 0.9835  
softplus activation, time 38 sec, accuracy 0.9836  
softplus activation, time 32 sec, accuracy 0.9819  
linear activation, time 27 sec, accuracy 0.9848  
linear activation, time 31 sec, accuracy 0.9854  
linear activation, time 31 sec, accuracy 0.9862  

Вывод: в этом эксперименте наилучшую точность дает tanh, второе место за relu.

#### Точность классификации в зависимости от числа полносвязных слоев


In [ ]:
for n_dense_layers in range(5):
  for i in range(2):
    acc, elapsed = train_model(mnist, get_model(n_dense_layers=n_dense_layers))
    print(f'{n_dense_layers} dense layers, time {elapsed:.0f} sec, accuracy {acc:.4f}')

0 dense layers, time 28 sec, accuracy 0.9864  
0 dense layers, time 32 sec, accuracy 0.9868  
1 dense layers, time 27 sec, accuracy 0.9897  
1 dense layers, time 28 sec, accuracy 0.9878  
2 dense layers, time 27 sec, accuracy 0.9899  
2 dense layers, time 29 sec, accuracy 0.9905  
3 dense layers, time 28 sec, accuracy 0.9898  
3 dense layers, time 32 sec, accuracy 0.9906  
4 dense layers, time 25 sec, accuracy 0.9882  
4 dense layers, time 29 sec, accuracy 0.9882  

Вывод: при 6 сверточных слоях добавление 2-3 полносвязных слоев повышает точность.

In [ ]:
total_layers = 10
for n_conv_layers in range(1, 11):
  n_dense_layers = total_layers - n_conv_layers
  acc, elapsed = train_model(mnist, get_model(n_conv_layers=n_conv_layers,
                                              n_dense_layers=n_dense_layers))
  print(f'{n_conv_layers} conv layers, {n_dense_layers} dense layers, time {elapsed:.0f} sec, accuracy {acc:.4f}')

1 conv layers, 9 dense layers, time 46 sec, accuracy 0.7561  
2 conv layers, 8 dense layers, time 39 sec, accuracy 0.9575  
3 conv layers, 7 dense layers, time 33 sec, accuracy 0.9812  
4 conv layers, 6 dense layers, time 33 sec, accuracy 0.9849  
5 conv layers, 5 dense layers, time 32 sec, accuracy 0.9886  
6 conv layers, 4 dense layers, time 27 sec, accuracy 0.9882  
7 conv layers, 3 dense layers, time 24 sec, accuracy 0.9893  
8 conv layers, 2 dense layers, time 33 sec, accuracy 0.9896  
9 conv layers, 1 dense layers, time 22 sec, accuracy 0.9871  
10 conv layers, 0 dense layers, time 30 sec, accuracy 0.9873  

Вывод: если суммарное количество слоев равно 10, то в этом эксперименте оптимально использовать 8 сверточных и 2 полносвязных слоя.

# CIFAR

#### Точность классификации в зависимости от числа сверточных слоев

In [ ]:
for n_conv_layers in range(1, 11):
  acc, elapsed = train_model(cifar, get_model(n_conv_layers=n_conv_layers))
  print(f'{n_conv_layers} conv layers, time {elapsed:.0f} sec, accuracy {acc:.4f}')

1 conv layers, time 203 sec, accuracy 0.3995
2 conv layers, time 323 sec, accuracy 0.5385
3 conv layers, time 180 sec, accuracy 0.5944
4 conv layers, time 179 sec, accuracy 0.6288
5 conv layers, time 122 sec, accuracy 0.6672
6 conv layers, time 136 sec, accuracy 0.6686
7 conv layers, time 109 sec, accuracy 0.6842
8 conv layers, time 98 sec, accuracy 0.6846
9 conv layers, time 118 sec, accuracy 0.7101
10 conv layers, time 106 sec, accuracy 0.7028


1 conv layers, time 203 sec, accuracy 0.3995  
2 conv layers, time 323 sec, accuracy 0.5385  
3 conv layers, time 180 sec, accuracy 0.5944  
4 conv layers, time 179 sec, accuracy 0.6288  
5 conv layers, time 122 sec, accuracy 0.6672  
6 conv layers, time 136 sec, accuracy 0.6686  
7 conv layers, time 109 sec, accuracy 0.6842  
8 conv layers, time 98 sec, accuracy 0.6846  
9 conv layers, time 118 sec, accuracy 0.7101  
10 conv layers, time 106 sec, accuracy 0.7028  

Вывод: увеличение количества сверточных слоев повышает точность (только на 10 слоях наблюдается падение).

#### Точность классификации в зависимости от числа каналов

In [ ]:
for n_filters in np.logspace(1, 2, num=10, dtype=int):
  acc, elapsed = train_model(cifar, get_model(n_filters=n_filters))
  print(f'{n_filters} filters, time {elapsed:.0f} sec, accuracy {acc:.4f}')

10 filters, time 263 sec, accuracy 0.5618
12 filters, time 143 sec, accuracy 0.5938
16 filters, time 258 sec, accuracy 0.6127
21 filters, time 133 sec, accuracy 0.6527
27 filters, time 128 sec, accuracy 0.6649
35 filters, time 151 sec, accuracy 0.6899
46 filters, time 134 sec, accuracy 0.7040
59 filters, time 129 sec, accuracy 0.7120
77 filters, time 196 sec, accuracy 0.7209
100 filters, time 208 sec, accuracy 0.7411


10 filters, time 263 sec, accuracy 0.5618  
12 filters, time 143 sec, accuracy 0.5938  
16 filters, time 258 sec, accuracy 0.6127  
21 filters, time 133 sec, accuracy 0.6527  
27 filters, time 128 sec, accuracy 0.6649  
35 filters, time 151 sec, accuracy 0.6899  
46 filters, time 134 sec, accuracy 0.7040  
59 filters, time 129 sec, accuracy 0.7120  
77 filters, time 196 sec, accuracy 0.7209  
100 filters, time 208 sec, accuracy 0.7411  

Вывод: увеличение количества каналов повышает точность.

#### Точность классификации в зависимости от типа пулинга

In [ ]:
for pool in ['max', 'avg']:
  for i in range(3):
    acc, elapsed = train_model(cifar, get_model(pool=pool))
    print(f'{pool} pooling, time {elapsed:.0f} sec, accuracy {acc:.4f}')

max pooling, time 182 sec, accuracy 0.6961
max pooling, time 150 sec, accuracy 0.6833
max pooling, time 176 sec, accuracy 0.6829
avg pooling, time 356 sec, accuracy 0.6921
avg pooling, time 550 sec, accuracy 0.7083
avg pooling, time 554 sec, accuracy 0.7100


max pooling, time 182 sec, accuracy 0.6961  
max pooling, time 150 sec, accuracy 0.6833  
max pooling, time 176 sec, accuracy 0.6829  
avg pooling, time 356 sec, accuracy 0.6921  
avg pooling, time 550 sec, accuracy 0.7083  
avg pooling, time 554 sec, accuracy 0.7100  

Вывод: в этом эксперименте average pooling оказался лучше max pooling.

#### Точность классификации в зависимости от функции активации

In [ ]:
for act_fn in ['relu', 'sigmoid', 'tanh', 'softplus', 'linear']:
  for i in range(3):
    acc, elapsed = train_model(cifar, get_model(act_fn=act_fn))
    print(f'{act_fn} activation, time {elapsed:.0f} sec, accuracy {acc:.4f}')

relu activation, time 151 sec, accuracy 0.6744
relu activation, time 169 sec, accuracy 0.6788
relu activation, time 160 sec, accuracy 0.6708
sigmoid activation, time 683 sec, accuracy 0.5342
sigmoid activation, time 614 sec, accuracy 0.5472
sigmoid activation, time 623 sec, accuracy 0.5057
tanh activation, time 204 sec, accuracy 0.6756
tanh activation, time 210 sec, accuracy 0.6870
tanh activation, time 218 sec, accuracy 0.6767
softplus activation, time 149 sec, accuracy 0.5442
softplus activation, time 217 sec, accuracy 0.5602
softplus activation, time 143 sec, accuracy 0.5424
linear activation, time 131 sec, accuracy 0.6648
linear activation, time 148 sec, accuracy 0.6646
linear activation, time 118 sec, accuracy 0.6685


relu activation, time 151 sec, accuracy 0.6744  
relu activation, time 169 sec, accuracy 0.6788  
relu activation, time 160 sec, accuracy 0.6708  
sigmoid activation, time 683 sec, accuracy 0.5342  
sigmoid activation, time 614 sec, accuracy 0.5472  
sigmoid activation, time 623 sec, accuracy 0.5057  
tanh activation, time 204 sec, accuracy 0.6756  
tanh activation, time 210 sec, accuracy 0.6870  
tanh activation, time 218 sec, accuracy 0.6767  
softplus activation, time 149 sec, accuracy 0.5442  
softplus activation, time 217 sec, accuracy 0.5602  
softplus activation, time 143 sec, accuracy 0.5424  
linear activation, time 131 sec, accuracy 0.6648  
linear activation, time 148 sec, accuracy 0.6646  
linear activation, time 118 sec, accuracy 0.6685  

Вывод: в этом эксперименте наилучшую точность дает relu, второе место за tanh.

#### Точность классификации в зависимости от числа полносвязных слоев


In [ ]:
for n_dense_layers in range(5):
  for i in range(2):
    acc, elapsed = train_model(cifar, get_model(n_dense_layers=n_dense_layers))
    print(f'{n_dense_layers} dense layers, time {elapsed:.0f} sec, accuracy {acc:.4f}')

0 dense layers, time 163 sec, accuracy 0.6653
0 dense layers, time 168 sec, accuracy 0.6750
1 dense layers, time 150 sec, accuracy 0.6981
1 dense layers, time 140 sec, accuracy 0.6874
2 dense layers, time 138 sec, accuracy 0.6890
2 dense layers, time 139 sec, accuracy 0.6970
3 dense layers, time 140 sec, accuracy 0.6924
3 dense layers, time 126 sec, accuracy 0.6772
4 dense layers, time 124 sec, accuracy 0.6624
4 dense layers, time 112 sec, accuracy 0.6609


0 dense layers, time 163 sec, accuracy 0.6653  
0 dense layers, time 168 sec, accuracy 0.6750  
1 dense layers, time 150 sec, accuracy 0.6981  
1 dense layers, time 140 sec, accuracy 0.6874  
2 dense layers, time 138 sec, accuracy 0.6890  
2 dense layers, time 139 sec, accuracy 0.6970  
3 dense layers, time 140 sec, accuracy 0.6924  
3 dense layers, time 126 sec, accuracy 0.6772  
4 dense layers, time 124 sec, accuracy 0.6624  
4 dense layers, time 112 sec, accuracy 0.6609  

Вывод: при 6 сверточных слоях добавление 2-3 полносвязных слоев повышает точность.

In [ ]:
total_layers = 10
for n_conv_layers in range(1, 11):
  n_dense_layers = total_layers - n_conv_layers
  acc, elapsed = train_model(cifar, get_model(n_conv_layers=n_conv_layers,
                                              n_dense_layers=n_dense_layers))
  print(f'{n_conv_layers} conv layers, {n_dense_layers} dense layers, time {elapsed:.0f} sec, accuracy {acc:.4f}')

1 conv layers, 9 dense layers, time 163 sec, accuracy 0.4239
2 conv layers, 8 dense layers, time 141 sec, accuracy 0.5547
3 conv layers, 7 dense layers, time 132 sec, accuracy 0.5866
4 conv layers, 6 dense layers, time 123 sec, accuracy 0.6116
5 conv layers, 5 dense layers, time 139 sec, accuracy 0.6468
6 conv layers, 4 dense layers, time 119 sec, accuracy 0.6642
7 conv layers, 3 dense layers, time 98 sec, accuracy 0.6803
8 conv layers, 2 dense layers, time 115 sec, accuracy 0.6942
9 conv layers, 1 dense layers, time 135 sec, accuracy 0.7001
10 conv layers, 0 dense layers, time 153 sec, accuracy 0.7060


1 conv layers, 9 dense layers, time 163 sec, accuracy 0.4239  
2 conv layers, 8 dense layers, time 141 sec, accuracy 0.5547  
3 conv layers, 7 dense layers, time 132 sec, accuracy 0.5866  
4 conv layers, 6 dense layers, time 123 sec, accuracy 0.6116  
5 conv layers, 5 dense layers, time 139 sec, accuracy 0.6468  
6 conv layers, 4 dense layers, time 119 sec, accuracy 0.6642  
7 conv layers, 3 dense layers, time 98 sec, accuracy 0.6803  
8 conv layers, 2 dense layers, time 115 sec, accuracy 0.6942  
9 conv layers, 1 dense layers, time 135 sec, accuracy 0.7001  
10 conv layers, 0 dense layers, time 153 sec, accuracy 0.7060  

Вывод: если суммарное количество слоев равно 10, то в этом эксперименте оптимально 10 полносвязных слоев